In [1]:
#Imports
import pandas as pd
import numpy as np
from patsy import dmatrices
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor

df = pd.read_csv('../data/DF_contact15_energy_sasa.skempiAB.Zero.csv')
df.dropna()
df = df._get_numeric_data() #drop non-numeric cols

df.head()


,covalent,vdw_clash,vdw,proximal,hbond,weak_hbond,ionic,aromatic,hydrophobic,carbonyl,...,if_phil,if_phob,if_total,sasa_phil_mut,sasa_phob_mut,sasa_total_mut,sasa_phil_wt,sasa_phob_wt,sasa_wt,ddG_exp
0,0.0,30.0,8.0,550.0,22.0,4.0,1.0,0.0,27.0,1.0,...,109.610,381.630,491.23,7734.47,23469.54,31204.02,7679.68,23535.64,31215.32,-0.209798
1,0.0,30.0,8.0,551.0,22.0,4.0,1.0,0.0,27.0,1.0,...,108.775,381.140,489.90,7688.39,23547.36,31235.76,7679.68,23535.64,31215.32,0.427282
2,0.0,30.0,8.0,551.0,22.0,4.0,1.0,0.0,27.0,1.0,...,113.565,377.240,490.80,7712.32,23498.20,31210.52,7679.68,23535.64,31215.32,0.466803
3,0.0,30.0,8.0,537.0,22.0,4.0,1.0,0.0,25.0,1.0,...,124.370,362.845,487.21,7701.35,23510.45,31211.80,7679.68,23535.64,31215.32,-0.032089
4,0.0,28.0,8.0,543.0,20.0,6.0,3.0,0.0,27.0,1.0,...,114.500,375.015,489.51,7703.32,23508.74,31212.06,7679.68,23535.64,31215.32,0.478730


In [2]:
scaler = StandardScaler()

In [3]:
#%%capture
#gather features
features = "+".join(df.columns.tolist()[:-1])

# get y and X dataframes based on this regression:
y, X = dmatrices('ddG_exp ~' + features, df, return_type='dataframe')

In [4]:
X_std = scaler.fit_transform(X)

In [5]:
# For each X, calculate VIF and save in dataframe
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X_std, i) for i in range(X.shape[1])]
vif["features"] = X.columns

/home/vilion/miniconda3/envs/bio/lib/python3.6/site-packages/statsmodels/regression/linear_model.py:1543: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/home/vilion/miniconda3/envs/bio/lib/python3.6/site-packages/statsmodels/stats/outliers_influence.py:181: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


In [6]:
vif["VIF Factor"].sort_values(ascending=False)

34             inf
59             inf
33             inf
35             inf
57             inf
58             inf
68    5.848831e+13
67    2.859428e+13
65    2.297755e+13
64    1.124494e+13
66    5.380645e+12
63    2.115856e+12
38    1.732347e+10
43    1.383718e+10
40    9.293610e+09
44    8.075953e+09
41    5.469035e+09
46    4.567344e+09
62    2.767616e+09
47    2.475790e+09
39    2.055482e+09
42    1.457851e+09
60    1.029548e+09
45    8.379018e+08
61    8.262862e+08
15    6.485085e+08
20    4.556587e+08
22    4.301476e+08
17    2.945890e+08
21    2.803175e+08
          ...     
26    4.041730e+06
30    3.168097e+06
51    2.529238e+06
28    4.975430e+05
31    3.192781e+05
25    1.384956e+05
50    5.573569e+01
37    4.664891e+01
11    4.347467e+01
5     3.652586e+01
56    2.978979e+01
4     2.920155e+01
7     2.076608e+01
2     2.007208e+01
8     1.786286e+01
9     1.743429e+01
36    1.603029e+01
3     1.122453e+01
1     1.010035e+01
12    9.823111e+00
6     5.818621e+00
10    5.7714

In [7]:
vif.loc[(vif["VIF Factor"] > 20)]

,VIF Factor,features
2,2.007208e+01,vdw_clash
4,2.920155e+01,proximal
5,3.652586e+01,hbond
7,2.076608e+01,ionic
11,4.347467e+01,polar
15,6.485085e+08,dg_change
16,3.464962e+07,backbone_hbond_change
17,2.945890e+08,sidechain_hbond_change
18,1.973687e+08,van_der_waals_change
19,4.509344e+07,electrostatics_change


In [45]:
for_drop = ["interface_residues_clashing_wt","partial_covalent_bonds_wt","entropy_mainchain_wt","backbone_hbond_wt",
           "electrostatics_wt","van_der_waals_clashes_wt"]
df.drop(for_drop,axis=1,inplace=True)

In [46]:
df.to_csv("../data/DF_contact400_energy_2019.f_corr2.csv",index=0)